In [1]:
import numpy as np
import os
import pandas as pd
from arcpy import env
from arcpy import TableToTable_conversion

dirWork = os.getcwd()
dirData = os.path.join(dirWork, r'data')
dirResults = os.path.join(dirWork, r'results')

print("Working Directory: " + dirWork)
print("Data Directory: " + dirData)
print("Results Directory: " + dirResults)

#define variables

dbf_Forecasts = os.path.join(dirData, r'StatewideForecasts_withUSTM_20201208.dbf')
csv_Forecasts = os.path.join(dirResults, r'forecasts_withUSTM.csv')
json_Forecasts = os.path.join(dirResults, r'forecasts_withUSTM.json')

# Create function that converts dbfs to csvs using arcpy
def dbfToCSV(dbf, csv):
    TableToTable_conversion(dbf, os.path.dirname(csv), os.path.basename(csv))

# Allow arcpy to overwrite output files
env.overwriteOutput = True

# convert dbfs to csvs
dbfToCSV(dbf_Forecasts, csv_Forecasts)

Working Directory: E:\GitHub\Traffic-Volume-Forecasts
Data Directory: E:\GitHub\Traffic-Volume-Forecasts\data
Results Directory: E:\GitHub\Traffic-Volume-Forecasts\results


In [2]:
df_Forecasts = pd.read_csv(csv_Forecasts)
df_Forecasts

,OID_,RELEASE,SEGID,ROUTE_ID,BMP,EMP,FULLNAME,CO_FIPS,CO_NAME,DISTANCE,...,AADT1989,AADT1988,AADT1987,AADT1986,AADT1985,AADT1984,AADT1983,AADT1982,AADT1981,PLANAREA
0,NaN,2019 RTP v8.3.1 20200508,0068_036.0,0068,36.037998,38.106998,CAMP WILLIAMS R,35,SALT LAKE,2.071000,...,1400,1500,1500,1400,1300,1200,1100,990,980,WFRC
1,NaN,2019 RTP v8.3.1 20200508,WFRC_8000,WFRC,1.488000,1.488000,,35,SALT LAKE,1.021000,...,0,0,0,0,0,0,0,0,0,WFRC
2,NaN,2019 RTP v8.3.1 20200508,0131_001.7,0131,1.742000,3.047000,PORTER ROCKWELL,35,SALT LAKE,1.298000,...,0,0,0,0,0,0,0,0,0,WFRC
3,NaN,2019 RTP v8.3.1 20200508,0068_038.1,0068,38.106998,39.167000,CAMP WILLIAMS R,35,SALT LAKE,1.061000,...,2000,2500,2400,2300,2100,1900,2000,0,0,WFRC
4,NaN,2019 RTP v8.3.1 20200508,2927_000.0,2927,0.000000,1.019000,DEER RIDGE DR,35,SALT LAKE,1.028000,...,0,0,0,0,0,0,0,0,0,WFRC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8113,NaN,,2565_001.6,2565,1.550000,1.693000,LOST CREEK RD,41,SEVIER,0.128480,...,0,0,0,0,0,0,0,0,0,UDOT
8114,NaN,,0091_003.7,91,3.744000,4.916000,HWY 91,3,BOX ELDER,1.166390,...,11000,9300,8900,8000,7700,7700,7300,7200,7300,WFRC
8115,NaN,,2822_000.0,2822,0.000000,3.328000,NEBO LOOP RD,49,UTAH,3.320740,...,0,0,0,0,0,0,0,0,0,MAG
8116,NaN,,0091_004.9,91,4.916000,5.738000,HWY 91,3,BOX ELDER,0.818745,...,11000,9300,8900,8000,7700,7700,7300,7200,7300,UDOT


In [3]:
meltIndex    = ['SEGID']
valueColumns = ['AADT2017','F2019','F2024','F2030','F2040','F2050']
df_Forecasts_melt = pd.melt(df_Forecasts, id_vars=meltIndex, value_vars=valueColumns)
df_Forecasts_melt.columns = ('SEGID','FCOL','ForecastAADT')
df_Forecasts_melt['FYEAR'] = (df_Forecasts_melt['FCOL'].str[-4:]).astype('int64')
df_Forecasts_melt = df_Forecasts_melt[['SEGID','FYEAR','ForecastAADT']]
df_Forecasts_melt.columns = ('S','Y','F')
df_Forecasts_melt = df_Forecasts_melt.sort_values(by=['S'])
df_Forecasts_melt

,S,Y,F
46830,0006_000.0,2050,600
22476,0006_000.0,2024,450
30594,0006_000.0,2030,500
38712,0006_000.0,2040,550
14358,0006_000.0,2019,450
...,...,...,...
43372,WFRC_8219,2050,13500
10900,WFRC_8219,2019,0
27136,WFRC_8219,2030,6200
35254,WFRC_8219,2040,6400


In [4]:
df_Forecasts_melt.to_json(json_Forecasts,orient='records')